# Google Cloud Parallel Data Read Speeds with Dask

This benchmarking was inspired by the [Abernathey et al. (2021)](https://www.computer.org/csdl/magazine/cs/2021/02/09354557/1reXu4gJjri) paper, which includes a detailed analysis and discussion of cloud-native big scientific data. It is recommended that a reader unacclimated with this subject matter first studies its information before going through this notebook.

Unfortunately, due to the access token required to retrieve data the Google Cloud Storage Bucket used in this benchmarking, the public reader will not be able to run this notebook on their own. If you wish to download your own copy and alter the code, the test data from [NOAA's ETOPO1 Global Relief Model](https://www.ngdc.noaa.gov/mgg/global/relief/ETOPO1/data/ice_surface/grid_registered/netcdf/) can be found here. A file transformation notebook is also found within the repository, which documents my methods for converting from the original NetCDF format. Note that a CSV version was created outside of this project space using Generic Mapping Tools (GMT). A validation notebook has also been included in the repository to test each file type for the correct data contents.

Different methods for accessing Google Cloud Storage are used--contigent on the file type of the data. Though discussed in further detail later on, I encourage the reader to read about the different file formats used in this benchmarking, as well as the access APIs. In many instances, data can be loaded directly into Dask objects without further manipulation. Other libraries were also used to load data in, mainly due to the limitations of Dask or speed benefits of using another library.

A significant portion of the code used in this benchmarking is directly taken from [Ryan Abernathey's demonstration notebook](https://github.com/earthcube2020/ec20_abernathey_etal/blob/master/cloud_storage.ipynb), and it is worth viewing before beginning this read.

 **NetCDF3, which is the original format of the ETOPO1_Ice_g_gmt4.nc file, does not support internal chunking. If you choose to download the data from the previously provided link, it must first be converted into NetCDF4 in order to successfully run the entire notebook.**

## Imports & Client Initialization

In [ ]:
import dask.array as dsa
import numpy as np
import dask.dataframe as dd
from contextlib import contextmanager
import xarray as xr
import intake # must install intake-xarray
import time
import dask
from matplotlib import pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.ticker import MaxNLocator
import pandas as pd
from scipy.stats import sem

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(threads_per_worker=2)
client = Client(cluster)

## Benchmarking Setup

First, we will create this null storage object. To measure our throughput, all of the data will need to be accessed at a single time and can be achieved by storing the data into this null storage target.

In [ ]:
class DevNullStore:
    def __init__(self):
        pass
    def __setitem__(*args, **kwargs):
        pass

null_store = DevNullStore()

The Diagnostic Timer will keep track of data retrieval times and store them within a Pandas DataFrame for later processing and analysis.

In [ ]:
class DiagnosticTimer:
    def __init__(self):
        self.diagnostics = []
        self.names = []
        
    @contextmanager
    def time(self, **kwargs):
        tic = time.time()
        yield
        toc = time.time()
        kwargs["runtime"] = toc - tic
        self.diagnostics.append(kwargs)
        
    def dataframe(self):
        return pd.DataFrame(self.diagnostics)
    
diag_timer = DiagnosticTimer()

This naming function will keep track of our read throughput for each file type and make it easier to plot all of the cases.

In [ ]:
def name(fileType, daf): # Takes a string fileType input & dataframe type input for daf 
    globals()[f"df_{fileType}"] = daf
    diag_timer.names.append(globals()[f"df_{fileType}"])
    
    global df, da
    del df, da
    
    diag_timer.diagnostics = []

Information about our cluster is also collected during the tests. Importantly, the number of parallel reads will correspond to the total number of workers.

In [ ]:
def total_nthreads():
    return sum([v for v in client.nthreads().values()])

def total_ncores():
    return sum([v for v in client.ncores().values()])

def total_workers():
    return len(client.ncores())

### Loop Selection

This is the main loop. A few modifications have been made from Abernathey's original version, but most notably we want to measure the connection time to the data reference. This shouldn't matter if we are purely measuring access times, but it is important when we use different modules to connect to Google Cloud Storage.

The user also has the choice between running the normal loop -- which calculates throughput a single time for each file format -- or a nested loop, which will run through each file type multiple times and determine the average throughput & error. The nested loop will take much longer to run, but will allow for a more accurate analysis.

**ONLY USE ONE OF THE FOLLOWING TWO SECTIONS AND RUN THE REST OF THE NOTEBOOK NORMALLY.**

#### Normal Loop

In [ ]:
def mainLoop(da, diag_kwargs):
    global max_workers, worker_step
    nworkers = max_workers
    while nworkers > 0:
        cluster.scale(nworkers)
        time.sleep(10)
        client.wait_for_workers(nworkers)
        print('Number of Workers:', nworkers)
        with diag_timer.time(nworkers=total_workers(), nthreads=total_nthreads(), ncores=total_ncores(), **diag_kwargs):
            future = dsa.store(da, null_store, lock=False, compute=False)
            dask.compute(future, retries=5)
        del future   
        nworkers -= worker_step
        
    df = diag_timer.dataframe()
    df['throughput_Mbps'] = da.nbytes / 1e6 / df['runtime']
    return df

#### Nested Loop

Choosing the nested loop requires that you run the `errorCalc(...)` function. It will keep track of the standard error of mean for each file type for use in error bars in later plots.

In [ ]:
tests = 5 # (integer) This number will correspond to the number of times each file format will be tested.

In [ ]:
def errorCalc(df):
    global tests
    errors = []
    means = []
    info = []
    sample = df['throughput_Mbps']
    for i in np.linspace(0, len(sample)-tests,int(len(sample)/tests), dtype='int'):
        temp = sample[i:(i+4)]
        errors.append(sem(temp))
        means.append(temp.mean())
        info.append(df.iloc[i, 0:7])
        
    df0 = pd.DataFrame(info, index=range(len(info)))
    df0['throughput_Mbps'] = pd.DataFrame(means)
    df0['errors'] = pd.DataFrame(errors)
    return df0

In [ ]:
def mainLoop(da, diag_kwargs):
    global tests, max_workers, worker_step
    nworkers = max_workers
    while nworkers > 0:
        cluster.scale(nworkers)
        time.sleep(10)
        client.wait_for_workers(nworkers)
        print('Number of Workers:', nworkers)
        for i in range(tests): #Change range
            with diag_timer.time(nworkers=total_workers(), nthreads=total_nthreads(), ncores=total_ncores(), **diag_kwargs):
                future = dsa.store(da, null_store, lock=False, compute=False)
                dask.compute(future, retries=5)
            del future   
        nworkers -= worker_step
        
    df = diag_timer.dataframe()
    df['throughput_Mbps'] = da.nbytes / 1e6 / df['runtime']
    df = errorCalc(df)
    return df

### Worker Range Input

Input the desired maximum workers and step size that the main loop will use to iterate. Note that they must be integer values.

In [ ]:
max_workers = 8 # (integer) This corresponds to the maximum amount of workers that will be used
worker_step = 1 # (integer) Will scale down the cluster by this amount through each iteration of the main loop
root = 'gs://cloud-data-benchmarks/'

## Perform Benchmarking

Though we are accessing the data from different formats, the core process will be the exact same. Dask is lazy by default, which means we will be using the previously defined null_store to measure throughput. Converting from DataFrames to Arrays, for example, will not affect the access speed because the data is not actually read from the source until we "store" it. So, these read speeds should be the same whether you are initially pointing to data using a dataframe or array.

### CSV

This is the classic tabular data format. Students & professionals alike use this format due to its simplicity and flexibility bewteen applications. As you will see, this format has the slowest read times of all being tested but is included in the benchmarking given its widespread use.

#### Single File

In [ ]:
tic1 = time.time()
df0 = dd.read_csv(root + 'ETOPO1_Ice_g_gmt4.csv',assume_missing=True, names=['lon', 'lat', 'z'])
toc1 = time.time()
connectTime = toc1-tic1

da = df0.to_dask_array(lengths=True)
del df0
chunksize = np.prod(da.chunksize) * da.dtype.itemsize
da

In [ ]:
diag_kwargs = dict(nbytes=da.nbytes, chunksize=chunksize, format='CSV', connectTime=connectTime)

df = mainLoop(da, diag_kwargs)
name('csv', df)
df_csv

#### Multiple Files

While one CSV file is the most common way one will see data of this format presented, we are interested in determining if partitioning (splitting up) the data into file sizes automatically determined by Dask will speed up the read from cloud storage. The process is the exact same, with the exception of a minor amount of required preprocessing.

In [ ]:
tic1 = time.time()
df0 = dd.read_csv(root + 'csvpartitions/*', assume_missing=True, names=['lon', 'lat', 'z'])
toc1 = time.time()
connectTime = toc1-tic1

da = df0.to_dask_array(lengths=True)
del df0
chunksize = np.prod(da.chunksize) * da.dtype.itemsize
da

In [ ]:
diag_kwargs = dict(nbytes=da.nbytes, chunksize=chunksize, format='Partitioned CSV', connectTime=connectTime)

df = mainLoop(da, diag_kwargs)
name('partcsv', df)
df_partcsv

### Parquet

Throughput will be tested in two different ways:
* **Multiple Files:** A CSV file was partitioned into a DataFrame by Dask, and 1 parquet file was written per DataFrame partition. All of these files will be read into the null storage target
* **Single File:** A single parquet file containing all of the original CSV information will be read into the null storage target

#### Multiple Files

In [ ]:
tic1 = time.time()
df0 = dd.read_parquet(root + 'parquetpartitions/*')
toc1 = time.time()
connectTime = toc1 - tic1

da = df0.to_dask_array(lengths=True)
del df0
chunksize = np.prod(da.chunksize) * da.dtype.itemsize
da

In [ ]:
diag_kwargs = dict(nbytes=da.nbytes, chunksize=chunksize, format='Partitioned Parquet', connectTime=connectTime)

df = mainLoop(da, diag_kwargs)
name('partparquet', df)
df_partparquet

#### Single File

We encounter a problem with large unmanaged memory when attempting to compute the size of DataFrame. This format is used here as an example of a file format that will not work in parallel, and should **only** be run to demonstrate a use case that does not function.

Generally, it is recommended that parquet files be partitioned in advance to avoid this issue. Dask currently creates DataFrame partitions based on the number of files stored within the parquet file path. Having one large file goes against the point of switching to parquet files in the first place because Dask will not be able to partition the data for parallel processing.

In [ ]:
tic1 = time.time()
df0 = dd.read_parquet(root + 'ETOPO1_Ice_g_gmt4.parquet')
toc1 = time.time()
connectTime = toc1 - tic1
# Memory Problem between these print statements, similar to the NetCDF issue
print("Calculating size of array")
da = df0.to_dask_array()
print("Size of array has been calculated")

del df0
chunksize = np.prod(da.chunksize) * da.dtype.itemsize
da

In [ ]:
diag_kwargs = dict(nbytes=da.nbytes, chunksize=chunksize, format='Parquet', connectTime=connectTime)

df = mainLoop(da, diag_kwargs)
name('parquet', df)
df_parquet

### NetCDF

NetCDF files are extremely common in geospatial and climate science, which is the original format of the test data used in this benchmarking. Data from these fields are commonly very large, with sizes only increasing as missions collecting samples at higher resolutions are conducted. The informed scientist should have a good grasp on the speed limitations of this format and the best ways to load in and perform computations within Python.

We load the NetCDF using the `intake` library. This is a great way to load the data from GCS without directly using the `gcsfs` module. Depending on the NetCDF file contents, decoding the data using the engines included with `xarray.open_dataset(...)` can be troublesome. Read the XArray documentation about accessing files using `xarray.open_dataset(...)` [here](https://docs.xarray.dev/en/stable/user-guide/io.html).

The `intake` module is much easier to use, and offers some additional functionality in the form of catalogs. Read about catalogs and their uses [here](https://intake.readthedocs.io/en/latest/catalog.html).

In [ ]:
tic1 = time.time()
data = intake.open_netcdf(root + 'ETOPO1_Ice_g_gmt4.nc').to_dask()
toc1 = time.time()
connectTime = toc1-tic1
data = data.to_array()
da = data.data # Retrieves raw values from wrapped Xarray DataArray object
da = dsa.from_array(da)
chunksize = np.prod(da.chunksize) * da.dtype.itemsize
del data
da

In [ ]:
diag_kwargs = dict(nbytes=da.nbytes, chunksize=chunksize, format='NetCDF', connectTime=connectTime)

df = mainLoop(da, diag_kwargs)
name('netcdf', df)
df_netcdf

### Zarr

Zarr Groups/Arrays are cloud-native gridded data formats, making them very useful for converting NetCDF files. It is known that there will be a large speedup with cloud read speed compared to the legacy data formats, but the performance differences between a Group/Array is important to note.

See <a href="https://www.programcreek.com/python/example/128207/dask.array.from_zarr#:~:text=.chunks)-,Example%205,-Project%3A%20napari">this python example</a> of `dask.array.from_zarr(...)` for further understanding of the differences between a Zarr Array & Zarr Group. A Zarr Group is a very good choice of storage utility for multiscale data due to its hierarchical paths in an object storage system -- this allows many different data sets to be stored in subdirectories under a larger Zarr store directory.

#### Zarr Array

In [ ]:
tic1 = time.time()
da = dsa.from_zarr(root + 'ETOPO1_Ice_g_gmt4.zarray')
toc1 = time.time()
connectTime = toc1 - tic1
chunksize = np.prod(da.chunksize) * da.dtype.itemsize
da

In [ ]:
diag_kwargs = dict(nbytes=da.nbytes, chunksize=chunksize, format='Zarr Array', connectTime=connectTime)

df = mainLoop(da, diag_kwargs)
name('zarray', df)
df_zarray

#### Zarr Hierachical Group

Xarray provides a fantasic tool in their API for opening Zarr groups. By consolidating the metadata upon creation of the group, the read speed increases considerably more than if it was left in its base state.

In [ ]:
tic1 = time.time()
zarr_ds = xr.open_zarr(store= root + 'ETOPO1_Ice_g_gmt4.zarr', consolidated=True)
toc1 = time.time()
connectTime = toc1-tic1

darray = zarr_ds.to_array()
da = darray.data
del zarr_ds, darray

chunksize = np.prod(da.chunksize) * da.dtype.itemsize
da

In [ ]:
diag_kwargs = dict(nbytes=da.nbytes, chunksize=chunksize, format='Zarr Group', connectTime=connectTime)

df = mainLoop(da, diag_kwargs)
name('zgroup', df)
df_zgroup

In [ ]:
client.close()
cluster.close()

## Plots

**If using exactly 4 different amounts of workers, the plot colors will enter a greyscale from. This is due to the color array containing length 4 for each file type, which matches the number of data points that will be plotted for each. In large scale testing this will not be a problem, but could be a problem when running this notebook on a local cluster.**

In [ ]:
def errorPlot(df, c):
    x = df['nworkers']
    y = df['throughput_Mbps']
    error = df['errors']
    plt.errorbar(x, y, yerr=error, color=c, fmt='o')

In [ ]:
color = cm.rainbow(np.linspace(0,1,len(diag_timer.names)))
legend = []

for i in range(len(diag_timer.names)):
    legend.append(diag_timer.names[i]['format'][1])
    c = color[i,:]
    
    if i==0:
        ax = diag_timer.names[i].plot(x='nworkers', y='throughput_Mbps', title='Cloud Data Read Speeds with Dask',
                                      kind='line', color=c, marker='o')
        try:
            errorPlot(diag_timer.names[i], c)
        except:
            pass
        else:
            errorPlot(diag_timer.names[i], c)
    
    elif i==(len(diag_timer.names)-1):
        diag_timer.names[i].plot(x='nworkers', y='throughput_Mbps', kind='line', color=c, ax=ax, 
                                 xlabel='Number of Parallel Reads', ylabel='Throughput (Mbps)', marker='o')
        try:
            errorPlot(diag_timer.names[i], c)
        except:
            print('Plotting Normally')
        else:
            print('Plotting with Error Bars')
            errorPlot(diag_timer.names[i], c)
        
        plt.grid(True)
        plt.legend(legend, bbox_to_anchor=[1.25, 0.5], loc='center', title='Store Formats')
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))
        #plt.yscale('symlog') ACTIVATE THIS LINE IF YOU ARE USING A LARGE AMOUNT OF WORKERS
    
    else:
        diag_timer.names[i].plot(x='nworkers', y='throughput_Mbps', kind='line', color=c, ax=ax, marker='o')
        try:
            errorPlot(diag_timer.names[i], c)
        except:
            pass
        else:
            errorPlot(diag_timer.names[i], c)